# Notes/Ideas



In [283]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib
from wordcloud import WordCloud
import pandas as pd
import numpy as np

import json

import requests

from bs4 import BeautifulSoup
import re

import time

import nltk
import string

plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('fivethirtyeight')
plt.style.use('ggplot')

In [284]:
import warnings
warnings.filterwarnings('ignore')

# Faculty Profiles data

In [285]:
df_faculty = pd.read_csv('../scraped_data/faculty_profiles.csv', sep='~')

In [286]:
df_faculty.head()

,faculty_name,faculty_profile_url,l_expertise,department,title_name,faculty_site_url,lab_url,faculty_email,description,description_links,...,link_to_news_3,description_teaser_3,article_date_4,title_of_news_4,link_to_news_4,description_teaser_4,article_date_5,title_of_news_5,link_to_news_5,description_teaser_5
0,David A. Aaker,/faculty/david-aaker,"business,marketing,branding",Haas School of Business,Professor of Marketing and Public Policy,http://www.haas.berkeley.edu/faculty/aaker.html,http://groups.haas.berkeley.edu/marketing/,aaker@haas.berkeley.edu,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pieter Abbeel,/faculty/pieter-abbeel,"robotics,machine learning",Division of Computer Science/EECS,Professor,http://www.cs.berkeley.edu/~pabbeel,NaN,pabbeel@cs.berkeley.edu,Robotics and Machine Learning.,NaN,...,/news/new-deep-learning-technique-enables-robo...,UC Berkeley researchers have developed algor...,"December 17, 2012",Big NSF grant funds research into training ro...,/news/big-nsf-grant-funds-research-training-ro...,"What if robots and humans, working together,...","August 23, 2011",UC Berkeley robotics expert named among world...,/news/uc-berkeley-robotics-expert-named-among-...,"Pieter Abbeel, a UC Berkeley, professor know..."
2,Elizabeth Abel,/faculty/elizabeth-abel,"feminist theory,psychoanalysis,Virginia Woolf,...",Department of English,Professor of English,http://english.berkeley.edu/profiles/5,NaN,eabel@uclink.berkeley.edu,Elizabeth Abel's general research interest is...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dor Abrahamson,/faculty/dor-abrahamson,"mathematical cognition,design-based research,m...",Graduate School of Education,Associate Professor of Cognition and Development,http://gse.berkeley.edu/people/dor-abrahamson,http://edrl.berkeley.edu/,dor@berkeley.edu,Dor Abrahamson studies the process of mathema...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Norman Abrahamson,/faculty/norman-abrahamson,"civil and environmental engineering,earthquake...",Department of Civil and Environmental Engineering,Adjunct Professor of Civil and Environmental E...,http://www.ce.berkeley.edu/faculty/faculty.php...,NaN,naa3@earthlink.net,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [287]:
df_faculty.columns.values

array(['faculty_name', 'faculty_profile_url', 'l_expertise', 'department',
       'title_name', 'faculty_site_url', 'lab_url', 'faculty_email',
       'description', 'description_links', 'article_date_1',
       'title_of_news_1', 'link_to_news_1', 'description_teaser_1',
       'article_date_2', 'title_of_news_2', 'link_to_news_2',
       'description_teaser_2', 'article_date_3', 'title_of_news_3',
       'link_to_news_3', 'description_teaser_3', 'article_date_4',
       'title_of_news_4', 'link_to_news_4', 'description_teaser_4',
       'article_date_5', 'title_of_news_5', 'link_to_news_5',
       'description_teaser_5'], dtype=object)

In [288]:
def split_first_last_name(s):
    """
    Find highest two 'words', those are the first and last name by order
    """
    l = s.lower().split(' ')
    lengths = [len(w) for w in l]
    max_index = max(range(len(l)), key=lambda i:lengths[i])
    max_word = l[max_index]
    lengths[max_index] = -1
    
    max_index2 = max(range(len(l)), key=lambda i:lengths[i])
    max_word2 = l[max_index2]
    
    if max_index < max_index2:
        return max_word, max_word2
    else:
        return max_word2, max_word
    
    

first_last_names = df_faculty['faculty_name'].apply(split_first_last_name)
first_names = [t[0] for t in first_last_names]
last_names = [t[1] for t in first_last_names]
df_faculty['first_name'] = first_names
df_faculty['last_name'] = last_names

In [289]:
df_faculty['full_name'] = df_faculty['first_name'] + ' ' + df_faculty['last_name']

In [290]:
bool_filter = ~df_faculty['l_expertise'].isnull()
df_faculty = df_faculty[bool_filter]
df_faculty['l_expertise'] = df_faculty['l_expertise'].apply(lambda s:s.lower())

In [291]:
len(set(df_faculty['department']))

72

In [292]:
set(df_faculty['department'])

{'Boalt Hall School of Law',
 'College of Engineering',
 'College of Letters and Science',
 'College of Natural Resources',
 'Department of African American Studies',
 'Department of Agricultural & Resource Economics',
 'Department of American Studies',
 'Department of Anthropology',
 'Department of Architecture',
 'Department of Art Practice',
 'Department of Astronomy',
 'Department of Bioengineering',
 'Department of Chemical Engineering',
 'Department of Chemistry',
 'Department of City & Regional Planning',
 'Department of Civil and Environmental Engineering',
 'Department of Classics',
 'Department of Comparative Literature',
 'Department of Demography',
 'Department of Earth and Planetary Science',
 'Department of East Asian Languages and Cultures',
 'Department of Economics',
 'Department of English',
 'Department of Environmental Science, Policy & Management',
 'Department of Ethnic Studies',
 'Department of Film and Media',
 'Department of French',
 "Department of Gender and 

In [324]:
finished_departments = set([
 'College of Engineering',
 'College of Letters and Science',
 'College of Natural Resources',
 'Department of Agricultural & Resource Economics',
 'Department of Astronomy',
 'Department of Bioengineering',
 'Department of Chemical Engineering',
 'Department of Chemistry',
 'Department of City & Regional Planning',
 'Department of Civil and Environmental Engineering',
 'Department of Economics',
 'Department of Environmental Science, Policy & Management',
 'Department of History',
 'Department of Industrial Engineering & Operations Research',
 'Department of Integrative Biology',
 'Department of Materials Science and Engineering',
 'Department of Mathematics',
 'Department of Molecular & Cell Biology',
 'Department of Nuclear Engineering',
 'Department of Nutritional Sciences & Toxicology',
 'Department of Physics',
 'Department of Plant and Microbial Biology',
 'Department of Statistics',
 'Division of Computer Science/EECS',
 'Division of Electribcal Engineering/EECS',
 'Environmental Science, Policy & Management',
 'Department of Earth and Planetary Science',
 'Division of Electrical Engineering/EECS',
 'Haas School of Business',
 'Helen Wills Neuroscience Institute',
 'Energy & Resources Group',
 'Department of Psychology',
 'Department of Political Science',
 'Department of Sociology',
 'Department of Anthropology',
 'Goldman School of Public Policy',
 'Department of Linguistics',
 'School of Public Health',
 'School of Social Welfare',
 'Landscape Architecture & Environmental Planning',
 'School of Information',
 'Department of Mechanical Engineering',
])

### Todo list

In [325]:
set(df_faculty['department']) - finished_departments

{'Boalt Hall School of Law',
 'Department of African American Studies',
 'Department of American Studies',
 'Department of Architecture',
 'Department of Art Practice',
 'Department of Classics',
 'Department of Comparative Literature',
 'Department of Demography',
 'Department of East Asian Languages and Cultures',
 'Department of English',
 'Department of Ethnic Studies',
 'Department of Film and Media',
 'Department of French',
 "Department of Gender and Women's Studies",
 'Department of Geography',
 'Department of German',
 'Department of History of Art',
 'Department of Italian Studies',
 'Department of Music',
 'Department of Near Eastern Studies',
 'Department of Philosophy',
 'Department of Rhetoric',
 'Department of Scandinavian',
 'Department of Slavic Languages and Literatures',
 'Department of South and Southeast Asian Studies',
 'Department of Spanish and Portuguese',
 'Graduate School of Education',
 'Interdisciplinary Studies (ISF)',
 'School of Journalism',
 'School of 

### Departments/Colleges/Institutes that are covered by smaller departments/mix of other departments

'Helen Wills Neuroscience Institute' is made up of mcb and other related departments

'Energy & Resources Group' espm, mcb, etc

### A lot of the faculty profiles don't work or don't point to the right source

In [308]:
df_faculty['faculty_site_url'].values[:30]

array(['http://www.haas.berkeley.edu/faculty/aaker.html',
       'http://www.cs.berkeley.edu/~pabbeel',
       'http://english.berkeley.edu/profiles/5',
       'http://gse.berkeley.edu/people/dor-abrahamson',
       'http://www.ce.berkeley.edu/faculty/faculty.php?id=780',
       'http://history.berkeley.edu/people/richard-m-abrams',
       'http://sph.berkeley.edu/barbara-abrams-drph-rd',
       'http://www.law.berkeley.edu/faculty/profiles/facultyProfile.php?facID=1105',
       'http://ced.berkeley.edu/ced/faculty-staff/charisma-acey', nan,
       'https://gspp.berkeley.edu/directories/faculty/dan-acland',
       'http://cci.lbl.gov/paul',
       'http://optometry.berkeley.edu/faculty/anthony-j-adams',
       'http://math.berkeley.edu/people/faculty/john-w-addison',
       'http://www.ieor.berkeley.edu/People/Faculty/adler.htm',
       'http://physics.berkeley.edu/people/faculty/mina-aganagic',
       'http://anthropology.berkeley.edu/users/sabrina-c-agarwal',
       'http://polisci.b

In [332]:
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}

def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element.encode('utf-8'))):
        return False
    elif element == '\n':
        return False
    return True

def scrape_urls(url, fn, base_url=''):
    if type(url) == type([]):
        url_l = url
        total_urls = {}
        for url in url_l:
            try:
                soup = BeautifulSoup(requests.get(url, headers=headers).content)
                urls = fn(soup)
                for faculty in urls:
                    total_urls[faculty] = base_url + urls[faculty]
            except:
                print(url)
        return total_urls

    soup = BeautifulSoup(requests.get(url, headers=headers).content)
    urls = fn(soup)
    for faculty in urls:
        urls[faculty] = base_url + urls[faculty]
    return urls

def feed_me_soup(soup, use_div, class_name=None, id_name=None):
    selector = 'div' if use_div else 'section'
    try:
        if id_name is not None:
            dump = list(filter(visible, soup.find_all(selector, {'id':id_name})[0].find_all(text=True)))
            return ' '.join([t for t in [t.replace(u'\xa0', u'').strip() for t in dump] if t != ''])
        dump = list(filter(visible, soup.find_all(selector, {'class':class_name})[0].find_all(text=True)))
        return ' '.join([t for t in [t.replace(u'\xa0', u'').strip() for t in dump] if t != ''])
    except:
        dump = list(filter(visible, soup.find_all(text=True)))
        return ' '.join([t for t in [t.replace(u'\xa0', u'').strip() for t in dump] if t != ''])

def scrape_faculty_page(url, department_name):
    soup = BeautifulSoup(requests.get(url, headers=headers).content)
    use_div = True
    if 'section' in departments_d[department_name]:
        use_div = False
        
    if 'use_content_id' in departments_d[department_name]:
        dump = feed_me_soup(soup, use_div, id_name=departments_d[department_name]['content_fn'])
    else:
        dump = feed_me_soup(soup, use_div, class_name=departments_d[department_name]['content_fn'])
    return dump

In [344]:
def get_me_urls(soup):
    urls = {}
    lis = soup.find_all('div', {'class':'view-content'})[0].find_all('ul')[0].find_all('li', recursive=False)
    for li in lis:
        a = li.find_all('span')[0].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_school_of_info_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[0].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_social_welfare_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'id':'block-system-main'})[0].find_all('div', {'class':'view-content'})[0]\
        .find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_public_health_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'id':'content'})[0].find_all('div', {'class':'view-content'})[0]\
        .find_all('div', recursive=False)
    for div in divs:
        block = div.find_all('div', recursive=False)
        name = block[1].find_all('h3')[0].text
        name = name.replace(',', '').replace('MD', '').replace('DrPH', '')\
                .replace('PhD', '').replace('MA', '')\
                .replace('MS', '').replace('MPH', '').replace('MRP', '').replace('RD', '').strip()
        try:
            url = block[3].find_all('a')[0]['href']
            urls[name] = url
        except:
            pass
    return urls

def get_linguistics_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'id':'main-inner'})[0].find_all('div', {'class':'panel-panel-inner'})[0]\
        .find_all('div', recursive=False)
    for div in divs:
        try:
            a = div.find_all('h2')[0].find_all('a')[0]
            url = a['href']
            if url.startswith('/'):
                url = 'http://lx.berkeley.edu{}'.format(url)
            urls[a.text] = url
        except:
            pass
    return urls

def get_anthro_urls(soup):
    urls = {}
    trs = soup.find_all('div', {'class':'view-content'})[1].find_all('table')[0].find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('td')[0].find_all('div')[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_sociology_urls(soup):
    urls = {}
    trs = soup.find_all('div', {'class':'view-content'})[0].find_all('table')[0].find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('td')[0].find_all('a')[0]
        urls[a.text] = a['href']

    return urls

def get_poli_sci_urls(soup):
    urls = {}
    trs = soup.find_all('div', {'class':'view-content'})[0].find_all('table')[0].find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('td')[0].find_all('a')[0]
        urls[a.text] = a['href']

    return urls

def get_psych_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[0].find_all('div', recursive=False)
    for div in divs:
        block = div.find_all('div', {'class':'panel-panel panel-col'})[0]
        a = block.find_all('a')[0]
        spans = block.find_all('span')
        urls['{} {}'.format(spans[0].text.strip(), spans[1].text.strip())] = a['href']
    return urls

def get_gssp_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'id':'content'})[0].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('h3')[0].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_haas_urls(soup):
    urls = {}
    blocks = soup.find_all('section', {'id':'center-col'})[0].find_all('div', {'class':'faculty-block'})
    for div in blocks:
        divs = div.find_all('div', recursive=False)
        for div in divs:
            a = div.find_all('p')[0].find_all('a')[0]
            last, first = split_first_last_name(a.text.replace('\n', '').replace('\t', '').replace(',', '').strip())
            urls['{} {}'.format(first, last)] = a['href']
    urls['john phillips'] = urls['(jack) phillips']
    del urls['(jack) phillips']
    return urls

def get_nuc_eng_urls(soup):
    urls = {}
    for a in soup.find_all('div', {'class':'view-content'})[0].find_all('a'):
        urls[a.text] = a['href']
    del urls['']
    return urls

def get_mse_urls(soup):
    urls = {}
    tables = soup.find_all('table', {'class':'facultytable'})
    for table in tables:
        tds = table.find_all('td')
        for td in tds:
            try:
                aas = td.find_all('a')
                if len(aas) > 1:
                    a = aas[1]
                else:
                    a = aas[0]
                urls[a.text.strip().replace('\xa0', ' ')] = a['href'].strip()
            except Exception as e:
                pass
    return urls

def get_eps_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div', recursive=False)
    for div in divs:
        ps = div.find_all('p')
        for p in ps:
            a = p.find_all('a')[0]
            urls[a.text] = a['href'][5:]
    return urls

def get_city_regional_planning_urls(soup):
    urls = {}
    lis = soup.find_all('div', {'id':'faculty'})[0].find_all('li')
    for li in lis:
        urls[li.find_all('h2')[0].text.replace('  ', ' ')] = li.find_all('a')[0]['href']
    return urls

def get_astro_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'row faculty-list researcher-list clearfix'})[0].find_all('div')
    for div in divs:
        a = div.find_all('a')[1]
        urls[a.text] = a['href']
    return urls

def get_plant_micro_bio_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[0].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_nutri_sci_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_physics_urls(soup):
    urls = {}
    uls = soup.find_all('div', {'class':'view-content'})[1].find_all('ul')
    for ul in uls:
        lis = ul.find_all('li')
        for li in lis:
            a = li.find_all('a')[1]
            urls[a.text] = a['href']
    return urls

def get_ib_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('a')[0]
        urls[a.text.replace('  ', ' ')] = a['href']
    return urls

def get_env_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text.replace('  ', ' ')] = a['href']
    return urls

def get_are_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'id':'content'})[0].find_all('div', recursive=False)[1:]
    for div in divs:
        subdivs = div.find_all('div', {'class':'view-content'})[0].find_all('div', recursive=False)
        for subdiv in subdivs:
            a = subdiv.find_all('div', recursive=False)[1].find_all('a')[0]
            urls[a.text] = a['href']
    return urls

def get_econ_urls(soup):
    urls = {}
    trs = soup.find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('td')[1].find_all('a')[0]
        last, first = a.text.strip().split(', ')
        urls[first + ' ' + last] = a['href']
    return urls

def get_chem_urls(soup):
    ahrefs = soup.find_all('div', {'class':'content'})[0]\
            .find_all('div', recursive='False')[0].find_all('div', recursive='False')[0]\
                .find_all('div', recursive='False')[0].find_all('a')
    urls = {}
    for a in ahrefs:
        try:
            if a['href'].startswith('http://'):
                last, first = a.text.split(', ')
                urls[first + ' ' + last] = a['href']
        except:
            pass
    return urls

def get_ieor_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'faculty-group clearfix'})
    for div in divs:
        lis = div.find_all('ol')[0].find_all('li', recursive=False)
        for li in lis:
            a = li.find_all('div')[2].find_all('a')[0]
            urls[a.text] = a['href']
    urls['J George Shanthikumar'] = urls[' \tJ George Shanthikumar ']
    del urls[' \tJ George Shanthikumar ']
    return urls

def get_civil_env_e_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div')
    for div in divs:
        try:
            a = div.find_all('div')[1].find_all('a')[0]
            urls[a.text.replace('  ', ' ')] = a['href']
        except:
            pass
    return urls

def get_bioe_urls(soup):
    urls = {}
    ahrefs = soup.find_all('div', {'class':'entry-content'})[0].find_all('a')
    for a in ahrefs:
        urls[a.find_all('strong')[0].text] = a['href']
    return urls

def get_mcb_urls(soup):
    urls = {}
    ahrefs = soup.find_all('div', {'id':'block-system-main'})[0].find_all('a')
    for a in ahrefs:
        try:
            url = a['href']
            if url.startswith('/'):
                urls[a.text] = url
        except:
            pass
    del urls['Faculty by Division']
    del urls['More Information >']
    return urls

def get_statistics_urls(soup):
    urls = {}
    tds = soup.find_all('tbody')[0].find_all('td')
    for td in tds:
        try:
            a = td.find_all('div')[2].find_all('a')
            urls[a[0].text] = a[0]['href']
        except:
            pass
    return urls

def get_math_urls(soup):
    urls = {}
    tds = soup.find_all('tbody')[0].find_all('td')
    for td in tds:
        try:
            a = td.find_all('div')[1].find_all('a')
            urls[a[0].text] = a[0]['href']
        except:
            pass
    return urls

def get_eecs_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'media'})
    for div in divs:
        a = div.find_all('a')
        urls[a[1].text] = a[0]['href']
    return urls

def get_history_urls(soup):
    urls = {}
    trs = soup.find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('a')[0]
        urls[a.text] = a['href']
    return urls

In [345]:
departments_d = {
    'history': {
        'url':'http://history.berkeley.edu/faculty',
        'base':'http://history.berkeley.edu',
        'url_fn':get_history_urls,
        'content_fn':'region region-content'
    },
    'eecs': {
        'url':'https://www2.eecs.berkeley.edu/Faculty/Lists/list.html?_ga=1.58368611.858746805.1441508156',
        'base':'https://www2.eecs.berkeley.edu',
        'url_fn':get_eecs_urls,
        'content_fn':'content'
    },
    'math': {
        'url':'https://math.berkeley.edu/people/faculty',
        'base':'https://math.berkeley.edu',
        'url_fn':get_math_urls,
        'content_fn':'content'
    },
    'stats': {
        'url':'http://statistics.berkeley.edu/people/faculty',
        'base':'http://statistics.berkeley.edu',
        'url_fn':get_statistics_urls,
        'content_fn':'content-wrapper',
        'use_content_id':True
    },
    'mcb': {
        'url':'https://mcb.berkeley.edu/faculty/all',
        'base':'https://mcb.berkeley.edu',
        'url_fn':get_mcb_urls,
        'content_fn':'content',
        'use_content_id':True
    },
    'bioe': {
        'url':'http://bioeng.berkeley.edu/people',
        'url_fn':get_bioe_urls,
        'content_fn':'entry-content'
    },
    'civil_env_e': {
        'url':'http://www.ce.berkeley.edu/people/faculty',
        'base':'http://www.ce.berkeley.edu',
        'url_fn':get_civil_env_e_urls,
        'content_fn':'view-content'
    },
    'ieor': {
        'url':'http://ieor.berkeley.edu/people/faculty',
        'base':'http://ieor.berkeley.edu',
        'url_fn':get_ieor_urls,
        'content_fn':'content',
        'use_content_id':True
    },
    'chem': {
        'url':'http://chemistry.berkeley.edu/faculty/chem',
        'url_fn':get_chem_urls,
        'content_fn':'content'
    },
    'cheme': {
        'url':'http://chemistry.berkeley.edu/faculty/cbe',
        'url_fn':get_chem_urls,
        'content_fn':'content'
    },
    'econ': {
        'url': 'https://www.econ.berkeley.edu/faculty',
        'url_fn':get_econ_urls,
        'content_fn':'view-content'
    },
    'are': {
        'url':'https://are.berkeley.edu/people/faculty',
        'base':'https://are.berkeley.edu',
        'url_fn':get_are_urls,
        'content_fn':'view-content'
    },
    'env': {
        'url':'https://ourenvironment.berkeley.edu/people/faculty-specialists',
        'base':'https://ourenvironment.berkeley.edu',
        'url_fn':get_env_urls,
        'content_fn':'view-content'
    },
    'ib': {
        'url':'http://ib.berkeley.edu/research/interests',
        'base':'https://ib.berkeley.edu',
        'url_fn':get_ib_urls,
        'content_fn':'node-content'
    },
    'astro_physics': {
        'url':'http://physics.berkeley.edu/research/astrophysics',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'molecular_physics': {
        'url':'http://physics.berkeley.edu/research/atomic-molecular-and-optical-physics',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'biophysics': {
        'url':'http://physics.berkeley.edu/research/biophysics',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'condensed_matter_physics': {
        'url':'http://physics.berkeley.edu/research/condensed-matter-physics-and-materials-science',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'nuclear_physics': {
        'url':'http://physics.berkeley.edu/research-faculty/nuclear-physics',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'particle_physics': {
        'url':'http://physics.berkeley.edu/research/particle-physics',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'plasma_physics': {
        'url':'http://physics.berkeley.edu/research/plasma-and-nonlinear-dynamics',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'nutri_sci': {
        'url':'http://nst.berkeley.edu/people/faculty',
        'base':'http://nst.berkeley.edu',
        'url_fn':get_nutri_sci_urls,
        'content_fn':'view-content'
    },
    'plant_micro_bio': {
        'url':'http://plantandmicrobiology.berkeley.edu/faculty/directory?_ga=1.25912820.858746805.1441508156',
        'base':'http://plantandmicrobiology.berkeley.edu',
        'url_fn':get_plant_micro_bio_urls,
        'content_fn':'entity entity-profile2 profile2-main clearfix'
    },
    'astro': {
        'url':'http://astro.berkeley.edu/people/researchers',
        'base':'http://astro.berkeley.edu',
        'url_fn':get_astro_urls,
        'content_fn':'detail-body'
    },
#     'city_regional_planning': {
#         'url':'https://ced.berkeley.edu/academics/city-regional-planning/people/faculty/',
#         'url_fn':get_city_regional_planning_urls,
#         'content_fn':'profile-content'
#     },
    'eps': {
        'url':'http://eps.berkeley.edu/research',
        'base':'http://eps.berkeley.edu/people',
        'url_fn':get_eps_urls,
        'content_fn':'ds-2col-stacked user-profile view-mode-full clearfix'
    },
    'mse': {
        'url':'https://www.mse.berkeley.edu/people/faculty',
        'url_fn':get_mse_urls,
        'content_fn':'entry-content'
    },
    'nuc_eng': {
        'url':'http://www.nuc.berkeley.edu/people/faculty',
        'base':'http://www.nuc.berkeley.edu',
        'url_fn':get_nuc_eng_urls,
        'content_fn':'content'
    },
    'haas': {
        'url':'http://facultybio.haas.berkeley.edu/faculty-photo/',
        'url_fn':get_haas_urls,
        'content_fn':'center-col',
        'use_content_id':True,
        'section':True
    },
    'gssp': {
        'url':'https://gspp.berkeley.edu/directories/faculty',
        'url_fn':get_gssp_urls,
        'content_fn':'content',
        'use_content_id':True,
    },
    'psych': {
        'url':'http://psychology.berkeley.edu/people/faculty',
        'base':'http://psychology.berkeley.edu/',
        'url_fn':get_psych_urls,
        'content_fn':'card-content',
    },
    'poli_sci': {
        'url':['http://polisci.berkeley.edu/people/faculty', 'http://polisci.berkeley.edu/people/faculty?page=1'],
        'base':'http://polisci.berkeley.edu',
        'url_fn':get_poli_sci_urls,
        'content_fn':'profile',
    },
    'sociology': {
        'url':'http://sociology.berkeley.edu/regular-faculty',
        'base':'http://sociology.berkeley.edu',
        'url_fn':get_sociology_urls,
        'content_fn':'profile',
    },
    'anthro': {
        'url':'http://anthropology.berkeley.edu/faculty',
        'base':'http://anthropology.berkeley.edu',
        'url_fn':get_anthro_urls,
        'content_fn':'user-profile-view',
        'use_content_id':True
    },
    'linguistics': {
        'url':'http://lx.berkeley.edu/people/faculty',
        'url_fn':get_linguistics_urls,
        'content_fn':'main-inner',
        'use_content_id':True
    },
    'public_health': {
        'url':['http://sph.berkeley.edu/faculty-profiles']+\
            ['http://sph.berkeley.edu/faculty-profiles?title=&page={}'.format(i) for i in range(1, 6)],
        'base':'http://sph.berkeley.edu',
        'url_fn':get_public_health_urls,
        'content_fn':'content',
        'use_content_id':True
    },
    'social_welfare': {
        'url':['http://socialwelfare.berkeley.edu/faculty-profiles',
            'http://socialwelfare.berkeley.edu/academic-researchers'],
        'base':'http://socialwelfare.berkeley.edu',
        'url_fn':get_social_welfare_urls,
        'content_fn':'block-system-main',
        'use_content_id':True
    },
    'ced': {
        'url':'https://ced.berkeley.edu/ced/faculty-staff/',
        'url_fn':get_city_regional_planning_urls,
        'content_fn':'profile-content'
    },
    'school_of_info': {
        'url':['https://www.ischool.berkeley.edu/people?name=&role=122'] + \
        ['https://www.ischool.berkeley.edu/people?name=&role=122&degr=All&year[value]&spec=All&emp=&page={}'.format(i)
            for i in range(1, 4)],
        'base':'https://www.ischool.berkeley.edu',
        'url_fn':get_school_of_info_urls,
        'content_fn':'main-content',
        'use_content_id':True
    },
    'me': {
        'url':'http://www.me.berkeley.edu/people/faculty',
        'base':'http://www.me.berkeley.edu',
        'url_fn':get_me_urls,
        'content_fn':'content',
        'use_content_id':True
    },
}

In [346]:
def get_urls(department):
    return departments_d[department]['urls']

In [347]:
for department in departments_d:
    d = departments_d[department]
    if 'base' in d:
        urls = scrape_urls(d['url'], d['url_fn'], d['base'])
    else:
        urls = scrape_urls(d['url'], d['url_fn'])
    d['urls'] = urls

In [348]:
text_d = {}
for department_name in departments_d:
    print('Starting: {}'.format(department_name))
    urls_d = get_urls(department_name)
    for faculty in urls_d:
        url = urls_d[faculty]
        try:
            text_d[faculty] = scrape_faculty_page(url, department_name)
        except Exception as e:
            print(department_name)
            print(e)

Starting: nutri_sci
Starting: molecular_physics
Starting: public_health
Starting: are
Starting: astro_physics
astro_physics
Exceeded 30 redirects.
Starting: math
Starting: astro
Starting: mcb
Starting: chem
Starting: nuc_eng
Starting: sociology
Starting: condensed_matter_physics
Starting: bioe
Starting: nuclear_physics
Starting: social_welfare
Starting: poli_sci
Starting: anthro
Starting: plasma_physics
Starting: mse
Starting: psych
Starting: ieor
Starting: econ
Starting: plant_micro_bio
Starting: linguistics
Starting: ced
Starting: particle_physics
Starting: haas
Starting: eps
Starting: env
Starting: ib
Starting: eecs
Starting: civil_env_e
Starting: cheme
Starting: stats
Starting: me
Starting: biophysics
Starting: school_of_info
Starting: history
Starting: gssp


In [349]:
rows = []
for faculty in text_d:
    rows.append([faculty, text_d[faculty]])
profile_df = pd.DataFrame(rows, columns=['faculty', 'text'])
profile_df.to_csv('../scraped_data/faculty_website_profiles.csv', sep='~', index=False)

In [350]:
len(profile_df)

2165

In [351]:
profile_df = pd.read_csv('../scraped_data/faculty_website_profiles.csv', sep='~')

## Ignore: for adding new departments/testing

In [233]:
set(df_faculty['department'])

{'Boalt Hall School of Law',
 'College of Engineering',
 'College of Letters and Science',
 'College of Natural Resources',
 'Department of African American Studies',
 'Department of Agricultural & Resource Economics',
 'Department of American Studies',
 'Department of Anthropology',
 'Department of Architecture',
 'Department of Art Practice',
 'Department of Astronomy',
 'Department of Bioengineering',
 'Department of Chemical Engineering',
 'Department of Chemistry',
 'Department of City & Regional Planning',
 'Department of Civil and Environmental Engineering',
 'Department of Classics',
 'Department of Comparative Literature',
 'Department of Demography',
 'Department of Earth and Planetary Science',
 'Department of East Asian Languages and Cultures',
 'Department of Economics',
 'Department of English',
 'Department of Environmental Science, Policy & Management',
 'Department of Ethnic Studies',
 'Department of Film and Media',
 'Department of French',
 "Department of Gender and 

In [340]:
gspp_url = 'https://gspp.berkeley.edu/directories/faculty'
psych_url = 'http://psychology.berkeley.edu/people/faculty'
base_url = 'http://psychology.berkeley.edu/'
poli_sci_url = 'http://polisci.berkeley.edu/people/faculty'
base = 'http://polisci.berkeley.edu'

sociology_url = 'http://sociology.berkeley.edu/regular-faculty'

anthro_url = 'http://anthropology.berkeley.edu/faculty'
linguistics_urls = 'http://lx.berkeley.edu/people/faculty'

public_health_urls = 'http://sph.berkeley.edu/faculty-profiles?title=&page=1'

"""http://sph.berkeley.edu/faculty-profiles?title=&page=1
http://sph.berkeley.edu/faculty-profiles?title=&page=2
http://sph.berkeley.edu/faculty-profiles?title=&page=3
http://sph.berkeley.edu/faculty-profiles?title=&page=5"""

social_welfare_urls = ['http://socialwelfare.berkeley.edu/faculty-profiles',
            'http://socialwelfare.berkeley.edu/academic-researchers']

school_of_info_urls = ['https://www.ischool.berkeley.edu/people?name=&role=122'] + \
        ['https://www.ischool.berkeley.edu/people?name=&role=122&degr=All&year[value]&spec=All&emp=&page={}'.format(i)
            for i in range(1, 4)]
    
school_of_optometry_urls = 'https://optometry.berkeley.edu/people/'

headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}
r = requests.get(public_health_urls, headers=headers)
soup = BeautifulSoup(r.content)

In [343]:
def get_sociology_urls(soup):
    urls = {}
    trs = soup.find_all('div', {'class':'view-content'})[0].find_all('table')[0].find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('td')[0].find_all('a')[0]
        urls[a.text] = a['href']

    return urls

def get_social_welfare_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'id':'block-system-main'})[0].find_all('div', {'class':'view-content'})[0]\
        .find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

#school of optometry hard codes data in javascript or either renders the data into javascript instead
# of directly rendering into html
# to make data miners lives more difficult?
#soup.find_all('article', {'class':'grid-people row'})[0].find_all('section')
#soup

In [49]:
urls

{'aaron bodoh-creed': 'http://facultybio.haas.berkeley.edu/faculty-list/bodohcreed-aaron/',
 'aaron mcdaniel': 'http://facultybio.haas.berkeley.edu/faculty-list/mcdaniel-aaron/',
 'abby scott': 'http://facultybio.haas.berkeley.edu/faculty-list/scott-abby/',
 'abhishek nagaraj': 'http://facultybio.haas.berkeley.edu/faculty-list/nagaraj-abhishek/',
 'adair morse': 'http://facultybio.haas.berkeley.edu/faculty-list/morse-adair/',
 'adam berman': 'http://facultybio.haas.berkeley.edu/faculty-list/berman-adam/',
 'adam leipzig': 'http://facultybio.haas.berkeley.edu/faculty-list/leipzig-adam/',
 'ajay bam': 'http://facultybio.haas.berkeley.edu/faculty-list/bam-ajay/',
 'alan cerf': 'http://facultybio.haas.berkeley.edu/faculty-list/cerf-alan/',
 'alan ross': 'http://facultybio.haas.berkeley.edu/faculty-list/ross-alan/',
 'alastair lawrence': 'http://facultybio.haas.berkeley.edu/faculty-list/lawrence-alastair/',
 'alexander nezlobin': 'http://facultybio.haas.berkeley.edu/faculty-list/nezlobin-al

In [60]:
faculty_url = 'http://facultybio.haas.berkeley.edu/faculty-list/berman-adam/'
r = requests.get(faculty_url, headers=headers)
soup = BeautifulSoup(r.content)

In [352]:
department = department_name = 'public_health'

d = departments_d[department]
if 'base' in d:
    urls = scrape_urls(d['url'], d['url_fn'], d['base'])
else:
    urls = scrape_urls(d['url'], d['url_fn'])
d['urls'] = urls

text_d = {}
urls_d = get_urls(department_name)
for faculty in urls_d:
    url = urls_d[faculty]
    try:
        text_d[faculty] = scrape_faculty_page(url, department_name)
    except Exception as e:
        print(department_name)
        print(e)

In [320]:
text_d.keys()

dict_keys(['John C. Neu', 'Nicolai  Reshetikhin', 'C. Keith Miller', 'Emiliano Gómez', 'Zhiqi Chen', 'Arthur E. Ogus', 'Michael Pejic', 'Weile Jia', 'John W. Addison Jr.', 'Leo Anthony Harrington', 'Alexandru Scorpan', 'Elwyn R. Berlekamp', 'Andrey Smirnov', 'Ming  Gu', 'Khoa L Nguyen', 'Mihaela Ifrim', 'Brian Krummel', 'Jenny  Harrison', 'Dana Scott', 'Robert M. Anderson', 'W. Hugh Woodin', 'Michael Viscardi', 'Tsit-Yuen  Lam (林節玄)', 'Chris  Shannon', 'Richard  Karp', 'William M. Kahan', 'Bernd  Sturmfels', 'Michele Schiavina', 'Khrystyna Serhiyenko', 'Peter Hintz', 'Per-Olof Persson', 'Alan Hammond', 'Joseph A. Wolf', 'Ole H. Hald', 'David  Eisenbud', 'John  Strain', 'Olga  Holtz', 'Ralph  McKenzie', 'James W. Demmel', 'Isadore M. Singer', 'Kenneth A. Ribet', 'James A. Sethian', 'Marina  Ratner', 'Pierre Simon', 'David Li-Bland', 'James  Pitman', 'Christian Blohmann', 'Clifton F. Ealy', 'Michael J. Klass', 'Takayuki Kihara', 'Alexander  Givental', 'Kelli  Talaska', 'Beresford N. Parl

In [326]:
text_d[list(text_d.keys())[0]]

'Professor Emeritus Research Primary Research Area: Applied Mathematics Additional Research Areas: Mathematical Analysis Research Interests: Applied mathematics Contact Information neu [at] math [dot] berkeley [dot] edu http://math.berkeley.edu/~neu/ Publications Selected Publications: Farjoun, Yossi and Neu, John (2008). An asymptotic solution of aggregation dynamics. In Progress in industrial mathematics at ECMI 2006 Math. Ind. 12 368-375  Springer Berlin.  [ MR ] [ GS? ] Bonilla, Luis and Carpio, Ana and Neu, John C. (2007). Kinetics of nucleation and growth: classical nucleation and helium bubbles in nuclear materials. In Math everywhere 291-307  Springer Berlin.  [ MR ] [ GS? ] Bonilla, L. L. and Carpio, A. and Neu, J. C. and Wolfer, W. G. (2006). Kinetics of helium bubble formation in nuclear materials. Phys. D 222 No.1-2,  131-140.  [ MR ] [ GS? ] Bonilla, L. L. and Carpio, A. and Farjoun, Y. and Neu, J. C. (2006). Asymptotic and numerical studies of the Becker-Döring model for 

### categories of text

- keywords/research fields
- description/bio
- selected publications

In [353]:
stopwords = set(nltk.corpus.stopwords.words('english') + list(string.punctuation))
common_dirty_words = set([
        'center-col',
        'id',
        'content',
        'img',
        'section',
        'page',
        'top', 'directory', 'icon', 'envelope', 'click',
        'navigation', 'www', 'com', 'html', 'link', 'http', 'edu', 'berkeley',
        'email', 'phone', 'room', 'download', 'pdf', 'california',
        'university', #remove this word or not?
        'printer-friendly', 'linkedin', 'profile', 'contact',
        'alphabetical', #maybe bad for linguistics
    ])

In [354]:
profile_df = pd.read_csv('../scraped_data/faculty_website_profiles.csv', sep='~')
profile_df.head()

,faculty,text
0,Anne Baranger,Director of Undergraduate Chemistry Faculty As...
1,Kathryn De Master,NaN
2,Solomon Hsiang,About Research In the News Areas of Expertise ...
3,Jasmina Vujic,About Welcome Message About the Department Dep...
4,George Leitmann,You are here Home › People › Faculty George Le...


In [384]:
def clean_text(text):
    try:
        text = text.lower()
        pattern = '[\r\t\n]'
        repl = ''
        text = re.sub(pattern, repl, text)

        pattern = '[=/\.~_\"“…*]'
        repl = ' '
        text = re.sub(pattern, repl, text)
        
        tokens = nltk.word_tokenize(text)

        l = []
        for token in tokens:
            if token in stopwords or token in common_dirty_words:
                continue
            if len(token) <= 2:
                continue
            contains_num = False
            for i in range(0, 10):
                if str(i) in token:
                    contains_num = True
                    break
            if contains_num:
                continue

            l.append(token)
        return l
    except:
        return []
    
cleaned_text = profile_df['text'].apply(clean_text)
profile_df['cleaned_text'] = cleaned_text

In [385]:
profile_df['cleaned_str_text']  = profile_df['cleaned_text'].apply(lambda l:' '.join(l))
cleaned_df = profile_df[['faculty', 'cleaned_str_text']]

In [386]:
cleaned_df.to_csv('../scraped_data/faculty_website_cleaned_profiles.csv', index=False)

In [387]:
cleaned_df.head()

,faculty,cleaned_str_text
0,Anne Baranger,director undergraduate chemistry faculty assis...
1,Kathryn De Master,
2,Solomon Hsiang,research news areas expertise agriculture clim...
3,Jasmina Vujic,welcome message department department history ...
4,George Leitmann,home people faculty george leitmann professor ...


In [388]:
df = pd.read_csv('../scraped_data/faculty_website_cleaned_profiles.csv')

In [389]:
df['cleaned_str_text'][0]

'director undergraduate chemistry faculty assistant teaching learning adjunct professor chemistry lecturer soe abaranger lewis hall educational resources website baranger research group research interests rna recognition proteins small molecules research program seeks understand control recognition rna proteins small molecules rna important steps gene expression usually acts concert proteins thus understanding recognition rna proteins small molecules necessary complete description biological processes involving rna development small molecules capable modulating processes one goal research determine proteins achieve high affinity specificity rna quantify rationally modulate energetic contributions systematically modify protein sequences rna nucleotides use computational fluorescence methods understand predict contributions energetic coupling dynamic processes recognition rna proteins principally studied rna recognition proteins containing rna recognition motif rrm one common rna-binding

In [238]:
np.isnan(profile_df['text'][6])

True

In [277]:
text = profile_df['text'][30].lower()
text

'professor of chemical and biomolecular engineering email: schaffer@berkeley.edu office: 274 stanley hall phone: 510.643.5963 fax: 510.642.4778 lab: 278 stanley hall lab phone: 510.642.4923 research group recent publications research interests biomedical engineering we have developed a research program that employs molecular and cellular engineering approaches to attempt to investigate biomedical problems. in particular, our lab is interested in the related areas of gene therapy and stem cell biology, with applications to therapies for diseases of the nervous system. biography professor. b.s. stanford university (1993); ph.d. massachusetts institute of technology (1998). distinguished lindsey lectureship, texas a&m (2012); eagleson award, american biological safety association (2012); fellow, american institute for medical and biological engineers (2010); robert w. vaughan lectureship, california institute of technology (2008); american chemical society biot division young investigator

In [278]:
pattern = '[\r\t\n]'
repl = ''
text = re.sub(pattern, repl, text)

pattern = '[=/\.~_\"“…*]'
repl = ' '
text = re.sub(pattern, repl, text)

In [279]:
tokens = nltk.word_tokenize(text)

l = []
for token in tokens:
    if token in stopwords or token in common_dirty_words:
        continue
    if len(token) <= 2:
        continue
    contains_num = False
    for i in range(0, 10):
        if str(i) in token:
            contains_num = True
            break
    if contains_num:
        continue
            
    l.append(token)

In [280]:
# l[len(l)//2:]
l

['professor',
 'chemical',
 'biomolecular',
 'engineering',
 'schaffer',
 'office',
 'stanley',
 'hall',
 'fax',
 'lab',
 'stanley',
 'hall',
 'lab',
 'research',
 'group',
 'recent',
 'publications',
 'research',
 'interests',
 'biomedical',
 'engineering',
 'developed',
 'research',
 'program',
 'employs',
 'molecular',
 'cellular',
 'engineering',
 'approaches',
 'attempt',
 'investigate',
 'biomedical',
 'problems',
 'particular',
 'lab',
 'interested',
 'related',
 'areas',
 'gene',
 'therapy',
 'stem',
 'cell',
 'biology',
 'applications',
 'therapies',
 'diseases',
 'nervous',
 'system',
 'biography',
 'professor',
 'stanford',
 'massachusetts',
 'institute',
 'technology',
 'distinguished',
 'lindsey',
 'lectureship',
 'texas',
 'eagleson',
 'award',
 'american',
 'biological',
 'safety',
 'association',
 'fellow',
 'american',
 'institute',
 'medical',
 'biological',
 'engineers',
 'robert',
 'vaughan',
 'lectureship',
 'institute',
 'technology',
 'american',
 'chemical',
 's

In [217]:
dump = scrape_faculty_page(urls[list(urls.keys())[i]], department_name)

In [226]:
text = nltk.Text(nltk.tokenize.word_tokenize(dump))

In [227]:
text.collocations()

lattice potentials; technological importance; Bose-Einstein
condensate; Physical Review; Review Letters; spinor Bose; atomic
physics; quantum physics


In [228]:
text.vocab()

FreqDist({"''": 2,
          '(': 15,
          ')': 15,
          ',': 62,
          '.': 36,
          '0.02': 1,
          '045305': 1,
          '108': 1,
          '11': 1,
          '143201': 1,
          '1486': 1,
          '1936': 1,
          '1999': 1,
          '2000': 2,
          '2001': 2,
          '2002': 2,
          '2003': 1,
          '2005': 1,
          '2006': 1,
          '2007': 2,
          '2012': 2,
          '2014': 1,
          '2015': 1,
          '312': 1,
          '344': 1,
          '433': 1,
          '720': 1,
          '95': 1,
          ':': 2,
          'A': 1,
          'APS': 1,
          'Alfred': 1,
          'America': 1,
          'American': 1,
          'And': 1,
          'Andrew': 1,
          'Another': 1,
          'Area': 1,
          'Ashvin': 1,
          'Atomic': 2,
          'Atoms': 1,
          'Award': 1,
          'Barrios': 1,
          'Berkeley': 1,
          'Biography': 1,
          'Bose': 2,
          'Bose-Einstein'

In [218]:
nltk.tokenize.word_tokenize(dump)

['Research',
 'Area',
 '(',
 's',
 ')',
 ':',
 'Atomic',
 ',',
 'Molecular',
 'and',
 'Optical',
 'Physics',
 'Condensed',
 'Matter',
 'Physics',
 'And',
 'Materials',
 'Science',
 'Biography',
 'Dan',
 'M.',
 'Stamper-Kurn',
 'came',
 'to',
 'Berkeley',
 'following',
 'his',
 'studies',
 'at',
 'the',
 'Massachusetts',
 'Institute',
 'of',
 'Technology',
 '(',
 'Ph.D.',
 '2000',
 ')',
 'and',
 'postdoctoral',
 'work',
 'at',
 'the',
 'California',
 'Institute',
 'of',
 'Technology',
 '(',
 '1999',
 '–',
 '2001',
 ')',
 '.',
 'He',
 'is',
 'the',
 'recipient',
 'of',
 'the',
 '2000',
 'APS',
 'Division',
 'of',
 'Atomic',
 ',',
 'Optical',
 'and',
 'Molecular',
 'Physics',
 'Outstanding',
 'Thesis',
 'award',
 ',',
 'the',
 'Alfred',
 'P.',
 'Sloan',
 'Fellowship',
 '(',
 '2001',
 '–',
 '2003',
 ')',
 ',',
 'the',
 'David',
 'and',
 'Lucile',
 'Packard',
 'Fellowship',
 'in',
 'Science',
 'and',
 'Engineering',
 '(',
 '2002',
 '–',
 '2007',
 ')',
 ',',
 'and',
 'the',
 'Presidential',


In [98]:
' '.join([t for t in [t.replace(u'\xa0', u'').replace("\'", "'").strip() for t in dump] if t != ''])

'Real name: Maureen C. Miller Professor Office Hours: Tuesday, 9-10; Thursday, 3-4 2219 Dwinelle (510) 642-5678 mcmiller@berkeley.edu Education: Ph.D., Harvard University, 1989 M.A. Catholic University of America, 1983 B.A. The American University, 1981 Curriculum Vitae: Miller_CV.pdf Research Interests: My research and teaching explores the extraordinary capacity of individuals and societies for change, using the rapid transformation of Europe over the eleventh and twelfth centuries as a lense to interogate contemporary understandings of life choices and the social, economic, political, and cultural forces conditioning them. Medieval religious beliefs and practices figure centrally in my work: I am profoundly interested in the radical claims made for human efficacy on the basis of otherworldly premises and the paradoxical results of attempts to realize religious ideals in this world. I have a particular interest in movements for reform in the eleventh and twelfth centuries. What reall

In [38]:
results = []
l_links_to_follow = []
count = 1
for url in urls:
    if count % 10 == 0:
        print(count)
    count+=1
    
    try:
        faculty_soup = BeautifulSoup(requests.get(base_url+url).content)
        #ahrefs = faculty_soup.find_all('a')
        #l = [ahref['href'] for ahref in ahrefs]
        #links_to_follow.append(l) #go one deep
        result = feed_me_soup(faculty_soup)
        results.append(result)
    except:
        results.append([])
        
    time.sleep(0.1)

10
20
30
40
50
60
70
80
90
100
110
120
130


In [39]:
results[1]

['Maneesh Agrawala',
 'Adjunct Professor',
 'Info Links',
 'Research Areas',
 'Human-Computer Interaction (HCI)',
 'Research Centers',
 'Berkeley Center for New Media (BCNM)',
 'Berkeley Vision and Learning Center (BVLC)',
 'Biography',
 "\n                        Maneesh Agrawala is an Adjunct Professor in Electrical Engineering and Computer Science at the University of California, Berkeley.  He is currently a Professor of Computer Science and Director of the Brown Institute for Media Innovation at Stanford University.  He works on visualization, computer graphics and human computer interaction. His focus is on investigating how cognitive design principles can be used to improve the effectiveness of visual displays. The goals of this work are to discover the design principles and then instantiate them in both interactive and automated design tools. He received an Okawa Foundation Research Grant in 2006, an Alfred P. Sloan Foundation Fellowship and an NSF CAREER Award in 2007, a SIGGRA